In [ ]:
import os

import pandas as pd
from markdown import markdown

In [ ]:
filename = 'suspect_library_5MSV_20200520.txt'
suspects = pd.read_csv(filename, sep='\t')

In [ ]:
suspects['dataset'] = suspects['SuspectPath'].str.slice(2, 14)
suspects['usi1'] = ('mzspec:' + suspects['dataset'] + ':' +
                    suspects['SuspectPath'].apply(os.path.basename) +
                    ':scan:' + suspects['SuspectScanNr'].astype(str))
suspects['usi2'] = 'mzspec:GNPSLIBRARY:' + suspects['LibraryID']
suspects['usi3'] = ('mzspec:MSV000084314:' + suspects['dataset'] +
                    '.mgf:scan:' + suspects['ClusterScanNr'].astype(str))

In [ ]:
output_list = ['Suspect | Mirror Library | Mirror Dataset Cluster | Image',
               '--- | --- | --- | ---']
suspects_str = (suspects['CompoundName'] + suspects['Mod'].map(
    {'addition': ' + ', 'loss': ' - '}) + suspects['PutativeID'])
mirror_urls = ('https://metabolomics-usi.ucsd.edu/svg/mirror?usi1=' +
               suspects['usi1'] + '&usi2=' + suspects['usi2'] +
               '&mz_min=50&mz_max=500')
mirror_alt_urls = ('https://metabolomics-usi.ucsd.edu/svg/mirror?usi1=' +
                   suspects['usi1'] + '&usi2=' + suspects['usi3'] +
                   '&mz_min=50&mz_max=500')
spectrum_urls = ('https://metabolomics-usi.ucsd.edu/svg/?usi=' +
                 suspects['usi1'] + '&mz_min=50&mz_max=500')
for sus, url1, url2, url3 in zip(
        suspects_str, mirror_urls, mirror_alt_urls, spectrum_urls):
    output_list.append(f'{sus} | ![]({url1}) | ![]({url2}) | [View USI]({url3})')

In [ ]:
with open(filename.replace('.txt', '.md'), 'w') as f_out:
    f_out.write('\n'.join(output_list))
with open(filename.replace('.txt', '.html'), 'w') as f_out:
    f_out.write(markdown('\n'.join(output_list), extensions=['tables']))